In [1]:
from collections import Counter

import pandas as pd

In [2]:
df = pd.read_csv("./data_cctweet1234.csv",sep='\t',encoding="utf-8")

In [3]:
df.head()

,CASE,SERIAL,REF,QUESTNNR,MODE,STARTED,I001_01,I002_CP,I002,I003_CP,...,Q004,Q005,FINISHED,Q_VIEWER,LASTPAGE,MAXPAGE,MISSING,MISSREL,TIME_RSI,DEG_TIME
0,21,NaN,5d16ab9b59da87001ac41a61,base,interview,2020-03-26 15:10:21,5d16ab9b59da87001ac41a61,0,9,0,...,3,3,1,0,13,13,0,0,1.01,5
1,23,NaN,5d566fa918f22f000148d118,base,interview,2020-03-26 15:11:03,5d566fa918f22f000148d118,0,18,0,...,3,3,1,0,13,13,0,0,1.42,53
2,24,NaN,5d553f1552d7670001906048,base,interview,2020-03-26 15:11:03,5d553f1552d7670001906048,0,17,0,...,2,3,1,0,13,13,0,0,1.27,29
3,25,NaN,5d46a0b7c46fcd001a238642,base,interview,2020-03-26 15:11:15,5d46a0b7c46fcd001a238642,0,3,0,...,3,3,1,0,13,13,0,0,0.78,3
4,26,NaN,5dd9bd7bf85b50953fdf003c,base,interview,2020-03-26 15:11:29,5dd9bd7bf85b50953fdf003c,0,8,0,...,2,3,1,0,13,13,0,0,0.92,7


In [4]:
def test_labels(row):
    right_answers = 0
    if row['Q001'] == 2:
        right_answers += 1
    if row['Q002'] == 2:
        right_answers += 1
    elif row['Q002'] == 3:
        right_answers += .5
    if row['Q003'] == 1:
        right_answers += 1
    if row['Q004'] == 3:
        right_answers += 1
    if row['Q005'] == 3:
        right_answers += 1
    return right_answers

df['validation'] = df.apply(lambda row: test_labels(row), axis=1)

In [5]:
df = df[df.validation >= 3.0]
data = df.loc[:, 'AA01':'AZ80']

In [6]:
data = data.apply(lambda x: x.astype('category'))


In [7]:
counts = data.apply(lambda column: column.value_counts()).fillna(0.0)

In [8]:
 list_of_counts = counts.values.T.tolist()
 list_of_labels = list(counts.columns.values)

In [9]:
def label_sort(values: """list w three values"""):
    if sum(values) < 3.0:
        return "NEEDS_ANNOTATION"
    elif (x := max(values)) < 3.0:
        return "unclear"
    elif values.count(x) > 1:
        indices = [i for i,v in enumerate(values) if v == x]
        if len(indices) == 3 or (0 in indices and 1 in indices):
            return "unclear"
        elif values[0] == 0 or (values[0]-values[1]) >= 2:
            return "denier"
        elif values[1] == 0 or (values[1]-values[0]) >= 2:
            return "believer"
        else:
            return "unclear"
    else:
        maximum = values.index(x)
        if maximum == 0:
            if (x-values[1]) >= 2:
                return "believer"
            else:
                return "unclear"
        elif maximum == 1:
            if (x-values[0]) >= 2:
                return "denier"
            else:
                return "unclear"
        elif maximum == 2:
            if (x-values[0]) >= 2 and (x-values[1]) >= 2:
                return "unclear"
            elif values[1]==0 or (values[0]-values[1]) > 3:
                return "believer"
            elif values[0]==0 or (values[1]-values[0]) > 3:
                return "denier"
            else:
                return "unclear"

In [11]:
labels = [label_sort(x) for x in list_of_counts]

In [12]:
labels = ["{}\n".format(x) for x in labels]
with open("annotations.txt","w+",encoding="utf-8") as f:
    f.writelines(labels)